# NP-Hard Problems

The purpose of this assignment is to familiarize yourself with different approaches to solving NP-hard problems in practice, especially via integer programming.

As in previous assignments, we will use [NetworkX](https://networkx.github.io/) to manipulate graphs and [PuLP](http://pythonhosted.org/PuLP/) to solve linear and integer programs.

In [1]:
import networkx as nx
import pulp

We will use two graph datasets for this assignment, both available in [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. The first is a well-known graph of the social network of a karate club [1]. The second is a network representing the western states power grid in the US [2].

In [2]:
karate = nx.read_gml('karate.gml',label='id')
power = nx.read_gml('power.gml',label='id')

[1] W. Zachary, An information flow model for conflict and fission in small groups, Journal of Anthropological Research 33, 452-473 (1977). 

[2] D. J. Watts and S. H. Strogatz, Collective dynamics of 'small world' networks, Nature 393, 440-442 (1998). 

## Independent Set

Recall that an *independent set* in a graph is a set of nodes such that no two nodes are adjacent (connected by an edge). Finding the maximum size of an independent set in a graph is an NP-hard problem, so there is no known polynomial-time algorithm to solve it.

### Problem 1

Provide an *integer programming* formulation for this problem here. The graph is denoted by $G = (V, E)$. Use a binary variable $x_i \in \{0,1\}$ for each node $i \in V$ (you do not need any other variables).

#### Solution:

TODO: formulate the integer program here

Each node in the graph ($x_0$ through $x_n$) will be considered a variable, with a possibility of being assigned either a 1 (meaning that that node is part of the Independent Set solution), or a 0 (meaning that that node is not part of the Independent Set solution). Because this is an Integer Program, the variables can only take on the value of 1 or 0- no float values in between. This integrality constraint exists for every node in the graph. Additionally, to assure that the nodes selected truly form an independent set, there is a constraint added to ensure that for every edge in the graph, both of its endpoint nodes are not part of the solution- either one of them or neither of them can be part of the solution (hence their sum being equal to at most 1). As this is a maximization problem, the goal is to maximize the number of nodes in the Independent Set, that is, to maximize the number of variables whose value is 1. Thus, the objective function is to maximize the sum of the $x_i$ from $0 \leq i \leq n$.

This Integer Program can be formulated as the below:


#### Variables: 
* $x_i$ $\in$ {0,1} for each node $i$ $\in$ $V$ such that:
\begin{equation}
  x_i =\begin{cases}
    1, & \text{if $x_i$ is in the independent set}\\
    0, & \text{if $x_i$ is not in the independent set}
  \end{cases}
\end{equation}

#### Constraints:
* $x_i$ $\in$ {0,1}, for every $i$ $\in$ $V$
* $x_i + x_j$ $\leq 1$, for every $(i,j)$ $\in$ $E$

#### Objective Function:
$max$ $\sum\limits_{i=0}^{n} x_i$

### Problem 2

Implement a function that solves the integer program given a graph as input.

In [3]:
def independent_set_ip(graph):
    """Computes a maximum independent set of a graph using an integer program.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, int)]) The IP solution as a list of node-value pairs.
        
    """  
    prob = pulp.LpProblem('Max_Independent_Set_IP', pulp.LpMaximize)
    
    
    # Define variables- each node is a variable, and it is either in the solution and gets a 1, or it is not and gets a 0
    ip_variables = pulp.LpVariable.dicts('ip_variables', graph.nodes(), 0, 1, cat = 'Integer')
    
    
    # Objective- maximize the sum of the 1's (get as many nodes with value of 1 indicating that they are part of the solution)
    node_list = []
    for node in graph.nodes():
        node_list.append(ip_variables[node])
    prob += sum(node_list)
       
        
    # Constraint- make sure that for each edge in the graph, both endpoints are not included
    for edge in graph.edges():
        prob += ip_variables[edge[0]] + ip_variables[edge[1]] <= 1
        
        
    # Solve the IP
    prob.solve()
      
        
    # Return the dictionary values for the nodes- either 0 or 1                                           
    solution_list = []
    for node in graph.nodes():
        sol = (int(node), int(ip_variables[node].value()))
        solution_list.append(sol)

        
    return solution_list


The following code outputs the size of the sets computed by your function.

In [4]:
def set_weight(solution):
    """Computes the total weight of the solution of an LP or IP for independent set.
    
    Args:
      - solution (list[int, float]): the LP or IP solution
    
    Returns:
      (float) Total weight of the solution
    
    """
    return sum(value for (node, value) in solution)

In [5]:
karate_ind_set = independent_set_ip(karate)
print("Size of karate set = {}".format(set_weight(karate_ind_set)))
power_ind_set = independent_set_ip(power)
print("Size of power set = {}".format(set_weight(power_ind_set)))

Size of karate set = 20
Size of power set = 2738


### Problem 3

Take the *linear programming relaxation* of your integer program and implement a function to solve it. This simply means that in your integer program, you should replace each constraint $x_i \in \{0,1\}$ with $0 \leq x_i \leq 1$.

In [6]:
def independent_set_lp(graph):
    """Computes the solution to the linear programming relaxation for the
    maximum independent set problem.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, float)]) The LP solution as a list of node-value pairs.
        
    """ 
    prob = pulp.LpProblem('Max_Independent_Set_LP', pulp.LpMaximize)
    
    
    # Define variables- each node is a variable, but relax the integrality constraint
    lp_variables = pulp.LpVariable.dicts('lp_variables', graph.nodes(), 0, 1)
               
            
    # Objective- maximize the sum of the 1's (get as many nodes with value of 1 indicating that they are part of the solution)
    node_list = []
    for node in graph.nodes():
        node_list.append(lp_variables[node])
    prob += sum(node_list)
    
    
    # Constraint- make sure that for each edge in the graph, both endpoints are not included
    for edge in graph.edges():
        prob += lp_variables[edge[0]] + lp_variables[edge[1]] <= 1 
    
    
    # Solve the LP
    prob.solve()
          
        
    # Return the dictionary values for the nodes                                         
    solution_list = []
    for node in graph.nodes():
        sol = (int(node), float(lp_variables[node].value()))
        solution_list.append(sol)

        
    return solution_list


Let's see how the LP solutions compares to those of the IPs.

In [7]:
karate_ind_set_relax = independent_set_lp(karate)
print("Value of karate set = {}".format(set_weight(karate_ind_set_relax)))
power_ind_set_relax = independent_set_lp(power)
print("Value of power set = {}".format(set_weight(power_ind_set_relax)))

Value of karate set = 20.5
Value of power set = 2757.99999999997


A heuristic way to convert a fractional solution to an independent set is as follows. For each node $i$, include the node in the set if $x_i > 1/2$, and discard it otherwise. This will yield a set of $\alpha$ nodes which have $\beta$ edges between them. By removing at most one node for each edge, this yields an independent set of size at least $\alpha - \beta$.

Implement this rounding procedure.

In [8]:
def round_solution(solution, graph):

    # Create a list of nodes to remove from the original graph
    # Use it in conjunction with the remove_nodes_from function in NetworkX to create the subgraph
    
    nodes_to_remove = []

    for i in solution:
        if i[1] <= 0.5:
            nodes_to_remove.append(i[0])
        
    subgraph = graph.copy()
    subgraph.remove_nodes_from(nodes_to_remove)
    
    return subgraph


The following function assesses the quality of the heuristic approach.

In [9]:
def solution_quality(rounded, optimal):
    """Computes the percent optimality of the rounded solution.
    
    Args:
      - rounded (nx.Graph): the graph obtained from rounded LP solution
      - optimal: size of maximum independent set

    """
    num_nodes = rounded.number_of_nodes() - rounded.number_of_edges()
    return float(num_nodes) / optimal

Let's check the quality of this approach compared to the optimal IP solutions.

In [10]:
karate_rounded = round_solution(karate_ind_set_relax, karate)
karate_quality = solution_quality(karate_rounded, set_weight(karate_ind_set))
print("Quality of karate rounded solution = {:.0f}%".format(karate_quality*100))

power_rounded = round_solution(power_ind_set_relax, power)
power_quality = solution_quality(power_rounded, set_weight(power_ind_set))
print("Quality of power rounded solution = {:.0f}%".format(power_quality*100))

Quality of karate rounded solution = 90%
Quality of power rounded solution = 95%
